In [1]:
import sys
import os
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import boto3

In [200]:
# Disable annoying and incorrect warning
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
matplotlib.style.use('seaborn')

In [44]:
# Set your AWS profile name, or create session using your AWS keys
boto3.setup_default_session(profile_name="goodit")

In [45]:
table = boto3.resource("dynamodb").Table("patron-arbitrage-orders-new")

In [61]:
response = table.scan()
result = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    result.extend(response['Items'])

In [62]:
len(result)

1020

In [223]:
df = pd.DataFrame(result)
df.head()

,quantity,rest_reply_raw_order,arbitrage_hash8,created_at,symbol,status,client_order_id,transaction_time,updated_at,order_side,fired_at,exchange,comment,order_id,price
0,0.00091800,"{'symbol': 'BTCUSDT', 'cummulativeQuoteQty': '...",24305052,1623250474674,BTCUSDT,FILLED,24305052_order_1,1623250474881,1623250475397,SELL,1623250474759,binance,,6382706713,34456.17000000
1,0.09610000,"{'symbol': 'BNBBUSD', 'cummulativeQuoteQty': '...",36220038,1623201023362,BNBBUSD,FILLED,36220038_order_3,1623201023651,0,BUY,1623201023415,binance,,681085654,345.41000000
2,33.63000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",31703392,1623250532045,BUSDUSDT,FILLED,31703392_order_2,1623250532170,1623250532540,SELL,1623250532070,binance,,197194258,0.98910000
3,34.36000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",27075858,1623239596660,BUSDUSDT,FILLED,27075858_order_2,1623239596788,1623239597167,BUY,1623239596673,binance,,197084281,1.00930000
4,35.72000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",96775028,1623211882279,BUSDUSDT,FILLED,96775028_order_1,1623211882385,1623211882558,BUY,1623211882295,binance,,196856458,1.00930000


In [224]:
df.groupby("status").count()[["quantity"]]

,quantity
status,
ERROR,4
FILLED,1016


In [225]:
def convert_ms_column_to_datetime(a_df, column):
    a_df[f"{column}_dt"] = pd.to_datetime(a_df[column].apply(int), unit='ms')
        

def convert_times_to_datetime(a_df):
    convert_ms_column_to_datetime(a_df, "created_at")
    convert_ms_column_to_datetime(a_df, "updated_at")
    convert_ms_column_to_datetime(a_df, "fired_at")
    convert_ms_column_to_datetime(a_df, "transaction_time")   

In [226]:
df.fired_at = df.fired_at.fillna(0)
convert_times_to_datetime(df)
df.head()

,quantity,rest_reply_raw_order,arbitrage_hash8,created_at,symbol,status,client_order_id,transaction_time,updated_at,order_side,fired_at,exchange,comment,order_id,price,created_at_dt,updated_at_dt,fired_at_dt,transaction_time_dt
0,0.00091800,"{'symbol': 'BTCUSDT', 'cummulativeQuoteQty': '...",24305052,1623250474674,BTCUSDT,FILLED,24305052_order_1,1623250474881,1623250475397,SELL,1623250474759,binance,,6382706713,34456.17000000,2021-06-09 14:54:34.674,2021-06-09 14:54:35.397,2021-06-09 14:54:34.759,2021-06-09 14:54:34.881
1,0.09610000,"{'symbol': 'BNBBUSD', 'cummulativeQuoteQty': '...",36220038,1623201023362,BNBBUSD,FILLED,36220038_order_3,1623201023651,0,BUY,1623201023415,binance,,681085654,345.41000000,2021-06-09 01:10:23.362,1970-01-01 00:00:00.000,2021-06-09 01:10:23.415,2021-06-09 01:10:23.651
2,33.63000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",31703392,1623250532045,BUSDUSDT,FILLED,31703392_order_2,1623250532170,1623250532540,SELL,1623250532070,binance,,197194258,0.98910000,2021-06-09 14:55:32.045,2021-06-09 14:55:32.540,2021-06-09 14:55:32.070,2021-06-09 14:55:32.170
3,34.36000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",27075858,1623239596660,BUSDUSDT,FILLED,27075858_order_2,1623239596788,1623239597167,BUY,1623239596673,binance,,197084281,1.00930000,2021-06-09 11:53:16.660,2021-06-09 11:53:17.167,2021-06-09 11:53:16.673,2021-06-09 11:53:16.788
4,35.72000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",96775028,1623211882279,BUSDUSDT,FILLED,96775028_order_1,1623211882385,1623211882558,BUY,1623211882295,binance,,196856458,1.00930000,2021-06-09 04:11:22.279,2021-06-09 04:11:22.558,2021-06-09 04:11:22.295,2021-06-09 04:11:22.385


In [227]:
df.groupby("status").count()

,quantity,rest_reply_raw_order,arbitrage_hash8,created_at,symbol,client_order_id,transaction_time,updated_at,order_side,fired_at,exchange,comment,order_id,price,created_at_dt,updated_at_dt,fired_at_dt,transaction_time_dt
status,,,,,,,,,,,,,,,,,,
ERROR,4,0,4,4,4,4,4,4,4,4,4,4,0,4,4,4,4,4
FILLED,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016


In [228]:
df.arbitrage_hash8 = df.arbitrage_hash8.apply(int)

In [229]:
df_counts = df[df.status.isin({"FILLED", "PARTIALLY_FILLED"})].groupby("arbitrage_hash8").count().reset_index().rename(columns={"quantity": "filled_orders"})
df = df.merge(df_counts[["arbitrage_hash8", "filled_orders"]], how="inner")

,quantity,rest_reply_raw_order,arbitrage_hash8,created_at,symbol,status,client_order_id,transaction_time,updated_at,order_side,fired_at,exchange,comment,order_id,price,created_at_dt,updated_at_dt,fired_at_dt,transaction_time_dt,filled_orders
0,0.00091800,"{'symbol': 'BTCUSDT', 'cummulativeQuoteQty': '...",24305052,1623250474674,BTCUSDT,FILLED,24305052_order_1,1623250474881,1623250475397,SELL,1623250474759,binance,,6382706713,34456.17000000,2021-06-09 14:54:34.674,2021-06-09 14:54:35.397,2021-06-09 14:54:34.759,2021-06-09 14:54:34.881,3
1,31.97000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",24305052,1623250474674,BUSDUSDT,FILLED,24305052_order_2,1623250474830,1623250475200,BUY,1623250474726,binance,,197192887,1.00920000,2021-06-09 14:54:34.674,2021-06-09 14:54:35.200,2021-06-09 14:54:34.726,2021-06-09 14:54:34.830,3
2,0.00091800,"{'symbol': 'BTCBUSD', 'cummulativeQuoteQty': '...",24305052,1623250474674,BTCBUSD,FILLED,24305052_order_3,1623250474853,1623250475320,BUY,1623250474763,binance,,2661917778,35157.91000000,2021-06-09 14:54:34.674,2021-06-09 14:54:35.320,2021-06-09 14:54:34.763,2021-06-09 14:54:34.853,3
3,0.09610000,"{'symbol': 'BNBBUSD', 'cummulativeQuoteQty': '...",36220038,1623201023362,BNBBUSD,FILLED,36220038_order_3,1623201023651,0,BUY,1623201023415,binance,,681085654,345.41000000,2021-06-09 01:10:23.362,1970-01-01 00:00:00.000,2021-06-09 01:10:23.415,2021-06-09 01:10:23.651,3
4,32.88000000,"{'symbol': 'BUSDUSDT', 'cummulativeQuoteQty': ...",36220038,1623201023362,BUSDUSDT,FILLED,36220038_order_2,1623201023624,0,BUY,1623201023422,binance,,196791182,1.00940000,2021-06-09 01:10:23.362,1970-01-01 00:00:00.000,2021-06-09 01:10:23.422,2021-06-09 01:10:23.624,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,0.09200000,"{'symbol': 'BNBUSDT', 'cummulativeQuoteQty': '...",97904284,1623211684990,BNBUSDT,FILLED,97904284_order_2,1623211685208,1623211685551,SELL,1623211685102,binance,,2429161201,336.26000000,2021-06-09 04:08:04.990,2021-06-09 04:08:05.551,2021-06-09 04:08:05.102,2021-06-09 04:08:05.208,3
1016,0.01283000,"{'symbol': 'ETHUSDT', 'cummulativeQuoteQty': '...",97904284,1623211684990,ETHUSDT,FILLED,97904284_order_3,1623211685183,1623211685501,BUY,1623211685075,binance,,4543113976,2458.92000000,2021-06-09 04:08:04.990,2021-06-09 04:08:05.501,2021-06-09 04:08:05.075,2021-06-09 04:08:05.183,3
1017,0.00109100,"{'symbol': 'BTCBUSD', 'cummulativeQuoteQty': '...",3457592,1623207651915,BTCBUSD,FILLED,3457592_order_3,1623207652062,1623207652369,SELL,1623207651972,binance,,2657368560,32531.51000000,2021-06-09 03:00:51.915,2021-06-09 03:00:52.369,2021-06-09 03:00:51.972,2021-06-09 03:00:52.062,3
1018,0.00109100,"{'symbol': 'BTCUSDT', 'cummulativeQuoteQty': '...",3457592,1623207651915,BTCUSDT,FILLED,3457592_order_2,1623207652059,1623207652327,BUY,1623207651969,binance,,6373550048,33159.99000000,2021-06-09 03:00:51.915,2021-06-09 03:00:52.327,2021-06-09 03:00:51.969,2021-06-09 03:00:52.059,3


In [298]:
df_filled = df[df.status == "FILLED"]
df_filled = df_filled[df_filled.filled_orders == 3]
len(df_filled)

1008

In [299]:
df_filled["fills"] = df_filled.rest_reply_raw_order.apply(lambda v: len(v.get("fills")))

Remove chains which have not all orders with a single fill (2 or more fills)

In [300]:
df_1_fill = df_filled[df_filled["fills"] == 1]
len(df_1_fill)

934

In [303]:
df_1_fill_counts = df_1_fill.groupby("arbitrage_hash8").count().reset_index().rename(columns={"quantity": "single_fill_orders"})
df_1_fill = df_1_fill.merge(df_1_fill_counts[["arbitrage_hash8", "single_fill_orders"]], how="inner")
df_1_fill = df_1_fill[df_1_fill.single_fill_orders == 3]
len(df_1_fill)

798

In [304]:
df_1_fill["fill_price"] = df_1_fill.rest_reply_raw_order.apply(lambda v: v.get("fills")[0].get("price")).apply(float)
df_1_fill["fill_volume"] = df_1_fill.rest_reply_raw_order.apply(lambda v: v.get("fills")[0].get("qty")).apply(float)
df_1_fill["fill_result_volume"] = df_1_fill["fill_price"] * df_1_fill["fill_volume"]

In [305]:
df_1_fill["fill_comission"] = df_1_fill.rest_reply_raw_order.apply(lambda v: v.get("fills")[0].get("commission"))
df_1_fill["fill_comission_coin"] = df_1_fill.rest_reply_raw_order.apply(lambda v: v.get("fills")[0].get("commissionAsset"))

In [306]:
from typing import Tuple
ARBITRAGE_COINS = {"USDT", "DOGE", "EUR", "BTC", "BUSD", "ETH", "BNB"}

def to_coins(market: str):
    for c in ARBITRAGE_COINS:
        if market.startswith(c):
            return c, market.replace(c, "")

def get_what_we_propose_volume(price, volume, side) -> Tuple[float, str]:
    return volume * price if side == "BUY" else volume

def get_what_we_get_volume(price, volume, side) -> Tuple[float, str]:
    return volume if side == "BUY" else volume * price

def get_what_we_propose_coin(market, side) -> str:
    coins = to_coins(market)
    return coins[1] if side == "BUY" else coins[0]

def get_what_we_get_coin(market, side) -> str:
    coins = to_coins(market)
    return coins[0] if side == "BUY" else coins[1]

In [308]:
df_1_fill["start_volume"] = df_1_fill.apply(lambda row: get_what_we_propose_volume(price=row.fill_price, volume=row.fill_volume, side=row.order_side), axis=1)
df_1_fill["start_coin"] = df_1_fill.apply(lambda row: get_what_we_propose_coin(market=row.symbol, side=row.order_side), axis=1)
df_1_fill["end_volume"] = df_1_fill.apply(lambda row: get_what_we_get_volume(price=row.fill_price, volume=row.fill_volume, side=row.order_side), axis=1)
df_1_fill["end_coin"] = df_1_fill.apply(lambda row: get_what_we_get_coin(market=row.symbol, side=row.order_side), axis=1)

In [349]:
def short_view(_df):
    return _df.sort_values(["arbitrage_hash8", "client_order_id"])[:21][["quantity", "arbitrage_hash8", "client_order_id", "symbol", "order_side", "fill_price", "fill_volume", "fill_result_volume", "start_volume", "start_coin", "end_volume", "end_coin", "profit"]]

In [434]:
def calc_profit(arbitrage_hash8):
    start_volume = df_1_fill.loc[df_1_fill.client_order_id == f"{arbitrage_hash8}_order_1"].start_volume
    end_volume = df_1_fill.loc[df_1_fill.client_order_id == f"{arbitrage_hash8}_order_3"].end_volume
    profit = float(end_volume) - float(start_volume)
    return profit

In [438]:
def calc_total_comission(arbitrage_hash8):
    order_1_com = df_1_fill.loc[df_1_fill.client_order_id == f"{arbitrage_hash8}_order_1"].fill_comission
    order_2_com = df_1_fill.loc[df_1_fill.client_order_id == f"{arbitrage_hash8}_order_2"].fill_comission
    order_3_com = df_1_fill.loc[df_1_fill.client_order_id == f"{arbitrage_hash8}_order_3"].fill_comission
    return float(order_1_com) + float(order_2_com) + float(order_3_com)

In [436]:
df_1_fill["profit"] = df_1_fill.apply(lambda row: calc_profit(row.arbitrage_hash8), axis=1)

In [439]:
df_1_fill["comission_sum"] = df_1_fill.apply(lambda row: calc_total_comission(row.arbitrage_hash8), axis=1)

### Calc profit in USD

In [440]:
short_view(df_1_fill)

,quantity,arbitrage_hash8,client_order_id,symbol,order_side,fill_price,fill_volume,fill_result_volume,start_volume,start_coin,end_volume,end_coin,profit
38,0.00616000,726952,726952_order_1,ETHUSDT,SELL,2521.18000,0.00616,15.530469,0.006160,ETH,15.530469,USDT,0.000008
40,0.04410000,726952,726952_order_2,BNBUSDT,BUY,353.39000,0.04410,15.584499,15.584499,USDT,0.044100,BNB,0.000008
39,0.04400000,726952,726952_order_3,BNBETH,SELL,0.14018,0.04400,0.006168,0.044000,BNB,0.006168,ETH,0.000008
275,0.01071000,828485,828485_order_1,ETHUSDT,SELL,2513.65000,0.01071,26.921192,0.010710,ETH,26.921192,USDT,0.000010
276,27.01000000,828485,828485_order_2,BUSDUSDT,BUY,0.99930,27.01000,26.991093,26.991093,USDT,27.010000,BUSD,0.000010
277,0.01072000,828485,828485_order_3,ETHBUSD,BUY,2515.93000,0.01072,26.970770,26.970770,BUSD,0.010720,ETH,0.000010
113,29.39000000,1612420,1612420_order_1,BUSDUSDT,SELL,0.99920,29.39000,29.366488,29.390000,BUSD,29.366488,USDT,-0.006715
112,0.01161000,1612420,1612420_order_2,ETHUSDT,BUY,2529.39000,0.01161,29.366218,29.366218,USDT,0.011610,ETH,-0.006715
111,0.01161000,1612420,1612420_order_3,ETHBUSD,SELL,2530.86000,0.01161,29.383285,0.011610,ETH,29.383285,BUSD,-0.006715
857,0.00762000,2201605,2201605_order_1,ETHEUR,SELL,2036.00000,0.00762,15.514320,0.007620,ETH,15.514320,EUR,0.000010


# Todo:
    0. Fold to chains
    1. Profit to USD
    2. Comission to USD
    3. Net profit USD per chain
    4. Net profit, total, with and without comission
    5. Merge with arbirages, to compare rois/profits

### Fold to chains

In [541]:
df_chains = df_1_fill[df_1_fill.client_order_id.str.endswith("_order_1")]
len(df_chains)

266

In [542]:
df_chains = df_chains[["arbitrage_hash8", "created_at_dt", "start_volume", "start_coin", "comission_sum", "profit"]]
df_chains = df_chains.sort_values("created_at_dt", ascending=False)
df_chains

,arbitrage_hash8,created_at_dt,start_volume,start_coin,comission_sum,profit
178,27740910,2021-06-09 15:06:04.320,0.000820,BTC,0.000122,0.000001
113,1612420,2021-06-09 15:05:32.053,29.390000,BUSD,0.000123,-0.006715
712,70054851,2021-06-09 15:03:19.706,0.028500,BNB,0.000043,0.000000
448,78844460,2021-06-09 15:03:19.370,21.150000,EUR,0.000163,-0.022778
842,87980585,2021-06-09 15:03:13.844,16.050000,EUR,0.000124,-0.007600
...,...,...,...,...,...,...
372,18769231,2021-06-09 00:04:56.399,37.030000,BUSD,0.000157,-0.013101
353,31588929,2021-06-09 00:01:42.208,37.044051,USDT,0.000158,0.046479
265,5334933,2021-06-09 00:01:09.657,0.014790,ETH,0.000158,0.000030
807,79393104,2021-06-08 23:53:01.655,36.952795,USDT,0.000157,0.047581


### Calc profit and comissions in USD

In [543]:
with open("usdt_prices.json", "r") as f:
    prices_usdt = json.load(f)

In [544]:
def to_usdt(volume, coin):
    if coin == "USDT":
        return volume
    return volume * prices_usdt.get(f"{coin}USDT")

In [545]:
df_chains["profit_usdt"] = df_chains.apply(lambda row: to_usdt(row.profit, row.start_coin) , axis=1)

In [546]:
total_profit_before_com = sum(df_chains.profit_usdt)
print(f"Total profit before comissions: ${total_profit_before_com}")

Total profit before comissions: $3.9264641511631195


In [547]:
df_chains["comission_sum_usdt"] = df_chains.apply(lambda row: float(row.comission_sum) * prices_usdt.get("BNBUSDT"), axis=1)

In [548]:
df_chains["profit_usdt_after_comission"] = df_chains.profit - df_chains.comission_sum_usdt

In [549]:
total_profit_after_com = sum(df_chains.profit_usdt_after_comission)
print(f"Total profit after comissions: ${total_profit_after_com}")

Total profit after comissions: $-11.37577239619989


In [550]:
df_chains["roi_before_com"] = df_chains.profit / df_chains.start_volume
df_chains["roi_after_com"] = df_chains.roi_before_com * df_chains.profit_usdt_after_comission / df_chains.profit_usdt

In [551]:
df_chains["start_volume_usdt"] = df_chains.apply(lambda row: to_usdt(row.start_volume, row.start_coin) , axis=1)

In [552]:
# df_chains.fillna(0, inplace=True)
df_chains

,arbitrage_hash8,created_at_dt,start_volume,start_coin,comission_sum,profit,profit_usdt,comission_sum_usdt,profit_usdt_after_comission,roi_before_com,roi_after_com,start_volume_usdt
178,27740910,2021-06-09 15:06:04.320,0.000820,BTC,0.000122,0.000001,0.036651,0.045335,-0.045334,0.001220,-0.001508,30.053681
113,1612420,2021-06-09 15:05:32.053,29.390000,BUSD,0.000123,-0.006715,-0.006711,0.045398,-0.052113,-0.000228,-0.001774,29.372366
712,70054851,2021-06-09 15:03:19.706,0.028500,BNB,0.000043,0.000000,0.000000,0.015824,-0.015824,0.000000,NaN,10.554120
448,78844460,2021-06-09 15:03:19.370,21.150000,EUR,0.000163,-0.022778,-0.027710,0.060310,-0.083088,-0.001077,-0.003229,25.728975
842,87980585,2021-06-09 15:03:13.844,16.050000,EUR,0.000124,-0.007600,-0.009245,0.045764,-0.053364,-0.000474,-0.002733,19.524825
...,...,...,...,...,...,...,...,...,...,...,...,...
372,18769231,2021-06-09 00:04:56.399,37.030000,BUSD,0.000157,-0.013101,-0.013093,0.058203,-0.071304,-0.000354,-0.001927,37.007782
353,31588929,2021-06-09 00:01:42.208,37.044051,USDT,0.000158,0.046479,0.046479,0.058362,-0.011884,0.001255,-0.000321,37.044051
265,5334933,2021-06-09 00:01:09.657,0.014790,ETH,0.000158,0.000030,0.077658,0.058636,-0.058606,0.002028,-0.001531,38.285394
807,79393104,2021-06-08 23:53:01.655,36.952795,USDT,0.000157,0.047581,0.047581,0.058240,-0.010659,0.001288,-0.000288,36.952795


# Report

In [553]:
df_chains_agg = df_chains[["profit_usdt", "comission_sum_usdt", "profit_usdt_after_comission", "roi_before_com", "roi_after_com", "start_volume_usdt"]]

In [554]:
df_chains_agg = df_chains_agg.rename(columns={
    "profit_usdt": "Profit, USDT",
    "comission_sum_usdt": "Comission (sum 3 orders), USDT",
    "profit_usdt_after_comission": "Profit - Comission, USDT",
    "roi_before_com": "ROI, Before Comissions",
    "roi_after_com": "ROI, After Comissions",
    "start_volume_usdt": "Trade Volume, in USDT"
})

In [555]:
df_chains_agg.describe(percentiles=[])

,"Profit, USDT","Comission (sum 3 orders), USDT","Profit - Comission, USDT","ROI, Before Comissions","ROI, After Comissions","Trade Volume, in USDT"
count,266.000000,266.000000,266.000000,266.000000,193.000000,266.000000
mean,0.014761,0.051098,-0.042766,0.000406,-0.001266,31.317513
std,0.264816,0.011751,0.048415,0.011610,0.001804,7.425978
min,-1.455038,0.015824,-0.193001,-0.096860,-0.005300,10.272804
50%,0.000000,0.053413,-0.052726,0.000000,-0.001518,34.236084
max,1.777536,0.081681,0.212772,0.072000,0.006452,42.427154


In [556]:
profitable_chains_percent = len(df_chains[df_chains.profit_usdt_after_comission > 0]) / len(df_chains)
print(f"Profitable chains percent:\t{100 * profitable_chains_percent:.2f}%")
print(f"Total profit BEFORE comissions:\t${total_profit_before_com:.2f}")
print(f"Total profit AFTER comissions:\t${total_profit_after_com:.2f}")
print(f"Total arbitrage runs analyzed:\t{len(df_chains)}")

Profitable chains percent:	12.03%
Total profit BEFORE comissions:	$3.93
Total profit AFTER comissions:	$-11.38
Total arbitrage runs analyzed:	266
